In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob

In [2]:
folder_store_Dom="/home/jian/BigLots/static_files/store_list_from_Dom/"
file_store_Dom=glob.glob(folder_store_Dom+"*.xlsx")
file_store_Dom=sorted(file_store_Dom,key=lambda x: os.stat(x).st_mtime)
file_store_Dom=file_store_Dom[-1]
file_store_Dom

'/home/jian/BigLots/static_files/store_list_from_Dom/Store List Report 07.10.20 425PM.xlsx'

In [3]:
excel_file=pd.ExcelFile(file_store_Dom)
df_store_type=excel_file.parse("Store List")
df_store_type=df_store_type[['Store','Furniture Type']].drop_duplicates()
df_store_type=df_store_type.rename(columns={"Store":"location_id"})
df_store_type=df_store_type[pd.notnull(df_store_type['location_id'])]
df_store_type['location_id']=df_store_type['location_id'].astype(str)

In [4]:
df_store_type.shape,df_store_type['location_id'].nunique()

((1479, 2), 1479)

In [5]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)
list_store_sales_files=list(recursive_file_gen("/home/jian/BigLots/"))

In [6]:
list_store_sales_files=[x for x in list_store_sales_files if "MediaStormSalesWeekly" in x]
list_store_sales_files=[x for x in list_store_sales_files if "_by_weeks/MediaStorm_" in x]



In [7]:
week_end_dt_2020Q2=[datetime.date(2020,8,1)-datetime.timedelta(days=7*i) for i in range(13)]
week_end_dt_2020Q1=[datetime.date(2020,5,2)-datetime.timedelta(days=7*i) for i in range(13)]

week_end_dt_2019Q2=[x-datetime.timedelta(days=7*52) for x in week_end_dt_2020Q2]
week_end_dt_2019Q1=[x-datetime.timedelta(days=7*52) for x in week_end_dt_2020Q1]

week_end_dt_2020Q2=[str(x) for x in week_end_dt_2020Q2]
week_end_dt_2020Q1=[str(x) for x in week_end_dt_2020Q1]
week_end_dt_2019Q2=[str(x) for x in week_end_dt_2019Q2]
week_end_dt_2019Q1=[str(x) for x in week_end_dt_2019Q1]

list_store_sales_files_2020Q2=[x for x in list_store_sales_files if x.split("_by_weeks/MediaStorm_")[1][:10] in week_end_dt_2020Q2]
list_store_sales_files_2020Q1=[x for x in list_store_sales_files if x.split("_by_weeks/MediaStorm_")[1][:10] in week_end_dt_2020Q1]
list_store_sales_files_2019Q2=[x for x in list_store_sales_files if x.split("_by_weeks/MediaStorm_")[1][:10] in week_end_dt_2019Q2]
list_store_sales_files_2019Q1=[x for x in list_store_sales_files if x.split("_by_weeks/MediaStorm_")[1][:10] in week_end_dt_2019Q1]

In [8]:
df_sales_by_store=pd.DataFrame()
for file in list_store_sales_files_2020Q2:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt']).drop_duplicates()
    df['year']="2020"
    df['quarter']="Q2"
    df_sales_by_store=df_sales_by_store.append(df)
for file in list_store_sales_files_2020Q1:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt']).drop_duplicates()
    df['year']="2020"
    df['quarter']="Q1"
    df_sales_by_store=df_sales_by_store.append(df)
for file in list_store_sales_files_2019Q2:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt']).drop_duplicates()
    df['year']="2019"
    df['quarter']="Q2"
    df_sales_by_store=df_sales_by_store.append(df)
for file in list_store_sales_files_2019Q1:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_sales_amt']).drop_duplicates()
    df['year']="2019"
    df['quarter']="Q1"
    df_sales_by_store=df_sales_by_store.append(df)
df_sales_by_store['gross_sales_amt']=df_sales_by_store['gross_sales_amt'].astype(float)

In [36]:
writer=pd.ExcelWriter("./BL_sales_by_furtype_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")

for quarter, df_group in df_sales_by_store.groupby("quarter"):
    df_week_count=df_group.groupby(["location_id"])['week_end_dt'].count().to_frame().reset_index()
    df_week_count_inclusion=df_week_count[df_week_count['week_end_dt']==26]
    list_store_inclusion=df_week_count_inclusion['location_id'].tolist()
    
    df_location_id_exclusion=df_week_count[~df_week_count['location_id'].isin(list_store_inclusion)]
    df_exclusion=df_group[~df_group['location_id'].isin(list_store_inclusion)]
    df_inclusion=df_group[df_group['location_id'].isin(list_store_inclusion)]

    df_output=pd.merge(df_inclusion,df_store_type,on="location_id",how="left")
    df_inclusion_stores=df_output[['location_id','Furniture Type','quarter']].drop_duplicates()
    df_output['Furniture Type']=df_output['Furniture Type'].fillna("nan")
    df_output['Furniture Type']=np.where(df_output['location_id']=="6990","Online",df_output['Furniture Type'])
    agg_func={'location_id':"nunique","gross_sales_amt":"sum"}
    df_output=df_output.groupby(['Furniture Type','year','quarter'])['location_id','gross_sales_amt'].agg(agg_func).reset_index()
    df_output=df_output.rename(columns={"gross_sales_amt":"sales"})
    df_output=df_output.pivot_table(index=['quarter','Furniture Type','location_id'],columns="year",values="sales")
    
    #### 
    df_output.to_excel(writer,"sales_%s_summary"%quarter,index=True)
    df_inclusion_stores.to_excel(writer,"IncludeStores_%s"%quarter,index=False)
    df_exclusion.to_excel(writer,"ExcludeStores_%s"%quarter,index=False)
    
writer.save()

    